### Bronze Notebook

In [0]:
# Databricks Widgets
dbutils.widgets.text(
    "source_path",
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",
    "Source CSV Path"
)

dbutils.widgets.text(
    "bronze_path",
    "/Volumes/workspace/ecommerce/ecommerce_data/delta/bronze/events",
    "Bronze Delta Path"
)

# Read widget values
source_path = dbutils.widgets.get("source_path")
bronze_path = dbutils.widgets.get("bronze_path")

# Read CSV
from pyspark.sql import functions as f
raw_df = spark.read.csv(source_path, header=True, inferSchema=True)

# Bronze Transformations
bronze_df = (
    raw_df
    .withColumn(
        "category_code",
        f.when(f.col("category_code").isNull(), "unknown")
         .otherwise(f.col("category_code"))
    )
    .withColumn(
        "product_name",
        f.regexp_extract(f.col("category_code"), r"([^.]+)$", 1)
    )
    .withColumn(
        "product_name",
        f.when((f.col("product_name") == "") | (f.col("product_name").isNull()), "unknown")
         .otherwise(f.col("product_name"))
    )
    .withColumn("ingestion_ts", f.current_timestamp())
    .withColumn("source_file", f.lit(source_path))
    .select(
        "event_time",
        "event_type",
        "product_id",
        "product_name",
        "category_id",
        "category_code",
        "brand",
        "price",
        "user_id",
        "user_session",
        "ingestion_ts",
        "source_file"
    )
)

# Write Bronze Delta
bronze_df.write.format("delta").mode("append").save(bronze_path)

# Optional: verify
display(bronze_df.limit(5))


event_time,event_type,product_id,product_name,category_id,category_code,brand,price,user_id,user_session,ingestion_ts,source_file
2019-11-01T00:00:00.000Z,view,1003461,smartphone,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33,2026-01-15T17:40:53.998Z,/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv
2019-11-01T00:00:00.000Z,view,5000088,sewing_machine,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283,2026-01-15T17:40:53.998Z,/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv
2019-11-01T00:00:01.000Z,view,17302664,unknown,2053013553853497655,unknown,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387,2026-01-15T17:40:53.998Z,/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv
2019-11-01T00:00:01.000Z,view,3601530,washer,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f,2026-01-15T17:40:53.998Z,/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv
2019-11-01T00:00:01.000Z,view,1004775,smartphone,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2,2026-01-15T17:40:53.998Z,/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv
